In [ ]:
from google.colab import drive
drive.mount('/content/drive')

root_path = '/content/drive/MyDrive/CVDL'
dumps_path =  f"{root_path}/dumps/"
dataset  = f"{root_path}/dataset/"

!cp "$root_path/modules/lib.py" .
!cp "$root_path/modules/plots.py" .
!cp "$root_path/modules/metrics.py" .

# Preprocessing

## Imports

In [ ]:
import cv2
import pickle
import matplotlib.pyplot as plt
import numpy as np
import cv2 as cv
import os
import tensorflow as tf
import importlib
from datetime import datetime

from keras.initializers import glorot_uniform

import lib
import plots
import metrics

tf.keras.backend.clear_session()

## Parameters

In [ ]:
# Deafault Res: 606x468 (ar 101 : 78)

# Tested Res's:
# 224x160 (ar 7 : 5)
# 384x288
# 576x432
# 512x384
input_res = (384, 288)

seed = 42

params = {
          'x':input_res[0],                                
          'y':input_res[1],                                
          'batch_size': 8,                        
          'n_channels_mask': 1,                            
          'n_channels': 1,                        
          'shuffle': True,              
          'learningRate': 1e-3,                      
          'momentum': 0.98,                        
          'nEpoches': 100,                        
          }

ratio_test = 0.85
ratio_val  = 0.8

lib.create_dump([params, seed], dumps_path + "params_du")

type_ds  = ['Images','Masks']
modality = ['train','validation','test']

# get path Img and Mask
path_img  = os.path.join(dataset, type_ds[0])
path_msk  = os.path.join(dataset, type_ds[1])
path_     = [path_img, path_msk]

# get length imgs and proof that is equal with masks
length_imgs  = len(os.listdir(os.path.join(dataset,type_ds[0] + "/"))) 
length_masks = len(os.listdir(os.path.join(dataset,type_ds[1] + "/")))
params['length_data'] = length_imgs,

# get name of file imgs
imgs_list_path = os.listdir(path_img)

## Data loading 

In [ ]:
# LOAD & SPLIT DATA ============================================================
mode = 1  # mode = 0: Random Split
          # mode = 1: Split for Patients
          
dataset_list = lib.get_dataset_list(
                                mode, 
                                path_, 
                                imgs_list_path, 
                                params, 
                                ratio_test, 
                                ratio_val, 
                                seed, 
                                modality)

# Dumping splitted data
lib.create_dump(dataset_list, f"{dumps_path}data_rt{ratio_test}_rv{ratio_val}_{input_res[0]}x{input_res[1]}")

##Loading Data from Pickle

In [ ]:
dataset_list = pickle.load(open( f"{dumps_path}data_rt{ratio_test}_rv{ratio_val}_{input_res[0]}x{input_res[1]}", "rb" ))

##Data Augmentation

In [ ]:
# DATA AUGMENTATION ============================================================
train_generator, val_generator = lib.data_augmentation(seed, dataset_list, params)

params["length_training"]   = len(dataset_list[0]['train'])
params['length_validation'] = len(dataset_list[0]['validation'])

augmented_batch_plot = plots.plot_augmented_batch(train_generator)
plt.show()

In [ ]:
print(params["length_training"])
print(params['length_validation'])
print(len(dataset_list[0]['test']))

# Model definition & training

## Training

In [ ]:
params['learningRate'] = 1e-3
params['nEpoches'] = 100


models_name = ['unet', 'lightweight_unet', 'dbunet', 'dbunet_BN']
arch_name = models_name[4]
model_weights = f"{root_path}/dataset/results/models/{arch_name}_Glorot_e{params['nEpoches']}_lr{params['learningRate']}"
log_path = f"{model_weights}/logs/fit/" + datetime.now().strftime("%Y%m%d-%H%M%S")
!rm -rf $log_path
!cp "$root_path/modules/models/{arch_name}.py" .
arch = importlib.import_module(arch_name)


input_size = (input_res[1], input_res[0], 1)
initializer = tf.keras.initializers.GlorotUniform()
model = arch.build_model(initializer, input_size)
model.summary()

In [ ]:
loss      = metrics.bce_dice()
recall    = tf.keras.metrics.Recall()
precision = tf.keras.metrics.Precision()
metric    = [metrics.dice, metrics.iou, recall, precision]

print(f'Model wil be saved in {model_weights}')

train_history, model = lib.train(model_weights, log_path, params, model, loss, metric, train_generator, val_generator)

## Evaluating

In [ ]:
results_tf = model.evaluate(dataset_list[0]['test'], dataset_list[1]['test'], return_dict = True)
train_figure = plots.plot_train_metric(train_history.history, params['nEpoches'], "dice")
plt.show()

In [ ]:
plt.imshow(dataset_list[1]['validation'][22, :,:, 0], cmap='gray')

In [ ]:
threshold = 0.5
min_dice = 0.60
results_df, avg = plots.get_results(dataset_list, params, model, threshold)
print(results_df)
print('\n')
print(f"Threshold value: {threshold}")
print(f"Best dice: {max(results_df['dice'])}")
print(f"Average dice: {avg}")
print(f"Minimum requested dice value (bad samples): {min_dice}")
good_res = results_df.loc[(results_df['dice'] >= min_dice)]
print(f"Average dice without bad samples: {good_res['dice'].mean()}")

In [ ]:
bad_res = results_df.loc[(results_df['dice'] < min_dice)]
print(bad_res)

num = bad_res['dice'].count()
den = results_df['dice'].count()

print(num/den)
print('\n')
print("Loading bad samples... :")
print('\n')
for idx in bad_res.index:
  print('\n')
  print('\n')
  print(f"Image index {idx} - dice: {bad_res.loc[idx]}")
  threshold = 0.5
  test_image = dataset_list[0]['test'][idx,:,:,0]
  test_mask = dataset_list[1]['test'][idx,:,:,0]
  test_results = lib.get_preditcions(test_image, test_mask, params, model, threshold)
  results_figure = plots.plot_test_results(test_results)

  plt.show()

In [ ]:
model.summary()

## Saving

In [ ]:
name = 'doubleunet_d0.9027'
model.save(f'/content/drive/MyDrive/CVDL/trained_models/{name}/model.h5', save_format='h5')
lib.create_dump(train_history.history, f'/content/drive/MyDrive/CVDL/trained_models/{name}/train_history')

In [ ]:
from tensorflow import keras

c_objects = {
    "bce_dice_loss": metrics.bce_dice_loss,
    "dice": metrics.dice,
    "iou": metrics.iou,
    "precision": tf.keras.metrics.Precision(),
    "recall": tf.keras.metrics.Recall(),
}

loaded_model = keras.models.load_model(f'/content/drive/MyDrive/CVDL/trained_models/{name}/model.h5', custom_objects=c_objects)

loaded_model.compile(loss=metrics.bce_dice_loss, optimizer='adam', metrics=metric)

results_tf = loaded_model.evaluate(dataset_list[0]['test'], dataset_list[1]['test'], return_dict = True)